In [1]:
# !pip install stable-baselines3
# !pip install pyglet
# !pip install swig
# !pip install box2d-py
# !pip install importlib-metadata==4.13.0
# !pip install optuna
# !pip install sb3_contrib

In [2]:
from stable_baselines3.common.env_util import make_vec_env
from sb3_contrib import TQC
from stable_baselines3.common.monitor import Monitor
import os
import gym
from stable_baselines3.common.evaluation import evaluate_policy
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances
from typing import Any, Dict
import torch
import torch.nn as nn
from stable_baselines3.common.callbacks import EvalCallback
import torch as th
import numpy as np
from easypip import easyimport
import functools
import time
easyimport("importlib_metadata==4.13.0")
OmegaConf = easyimport("omegaconf").OmegaConf
bbrl_gym = easyimport("bbrl_gym")
bbrl = easyimport("bbrl>=0.1.6")
import os
import copy
import time
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
from torch.distributions.normal import Normal
from torch.distributions.independent import Independent
import gym
from bbrl.agents.agent import Agent
from bbrl import get_arguments, get_class, instantiate_class
from bbrl.workspace import Workspace
from bbrl.agents import Agents, RemoteAgent, TemporalAgent
from bbrl.agents.gymb import AutoResetGymAgent, NoAutoResetGymAgent
from bbrl.visu.play import load_agent, play
from bbrl.utils.replay_buffer import ReplayBuffer
from bbrl.utils.functionalb import gae

# !rm -rf gym-rocketlander
# !git clone https://github.com/EmbersArc/gym-rocketlander
# !conda uninstall gym-rocketlander
# !pip install -e ./gym-rocketlander

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
class RocketLanderWrapper(gym.Wrapper):
    """
    Specific wrapper to shape the reward of the rocket lander environment
    """
    def __init__(self, env):
        super(RocketLanderWrapper, self).__init__(env)
        self.env = env
        self.prev_shaping = None
        
    def reset(self):
        self.prev_shaping = None
        return self.env.reset()

    def step(self, action):
        d = 1
        next_state, reward, done, info = self.env.step(action)
        # reward shaping
        """
        shaping = -0.5 * (self.env.distance + self.env.speed + abs(self.env.angle) ** 2)
        shaping += 0.1 * (
            self.env.legs[0].ground_contact + self.env.legs[1].ground_contact
        )
        if self.prev_shaping is not None:
            reward += shaping - self.prev_shaping
        self.prev_shaping = shaping
        """
        # print ("distance", self.env.distance)
        
        # shaping = 0.02
        shaping = 0.008 * (1 - self.env.distance)
        # shaping = 0.1 * (self.env.groundcontact - self.env.speed)
        if (
            self.env.legs[0].ground_contact > 0
            and self.env.legs[1].ground_contact > 0
            and self.env.speed < 0.1
        ):
            d = d * 2
            print("landed !")
            print ("speed", self.env.speed)
            shaping += 6.0 * d / self.env.speed
        else:
          d = 1
        reward += shaping
        return next_state, reward, done, info

    def old_step(self, action):
        next_state, reward, done, info = self.env.step(action)
        # reward shaping
        # shaping = -0.5 * (self.env.distance + self.env.speed + abs(self.env.angle) ** 2)
        # shaping += 0.1 * (self.env.legs[0].ground_contact + self.env.legs[1].ground_contact)
        shaping = 0
        if self.prev_shaping is not None:
            reward += shaping - self.prev_shaping
        self.prev_shaping = shaping

        return next_state, reward, done, info


class FrameSkip(gym.Wrapper):
    """
    Return only every ``skip``-th frame (frameskipping)
    :param env: the environment
    :param skip: number of ``skip``-th frame
    """

    def __init__(self, env: gym.Env, skip: int = 1):
        super().__init__(env)
        self._skip = skip

    def step(self, action: np.ndarray):
        """
        Step the environment with the given action
        Repeat action, sum reward, and max over last observations.
        :param action: the action
        :return: observation, reward, done, information
        """
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break

        return obs, total_reward, done, info

    def reset(self):
        return self.env.reset()

In [4]:
env = gym.make("RocketLander-v0")
env = RocketLanderWrapper(env)
env = FrameSkip(env, skip=4)
obs_space = env.observation_space
action_space = env.action_space
print("The observation space: {}".format(obs_space))
print("The action space: {}".format(action_space))

Matplotlib backend: module://matplotlib_inline.backend_inline
The observation space: Box([ -1.  -1.  -1.  -1.  -1.  -1.  -1. -inf -inf -inf], [ 1.  1.  1.  1.  1.  1.  1. inf inf inf], (10,), float32)
The action space: Box([-1. -1. -1.], [1. 1. 1.], (3,), float32)


In [14]:
# model = TQC("MlpPolicy", env, verbose=1,  gamma=0.99, 
#                                           learning_rate=0.001, 
#                                           policy_kwargs={"net_arch":[256, 256], 
#                                                          "activation_fn":nn.ReLU}
#                                           )
# model = TQC("MlpPolicy", env, verbose=1, n_eval_episodes=30, eval_freq=20000, log_interval=10, eval_env=10, seed=2182406907)

# model = TQC("MlpPolicy", env, verbose=1, seed=2182406907)

model.learn(total_timesteps=200_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 216      |
|    ep_rew_mean     | 1.04     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 46       |
|    time_elapsed    | 18       |
|    total_timesteps | 862      |
| train/             |          |
|    actor_loss      | -0.737   |
|    critic_loss     | 0.000223 |
|    ent_coef        | 0.00032  |
|    ent_coef_loss   | 1.63     |
|    learning_rate   | 0.0003   |
|    n_updates       | 200661   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 233      |
|    ep_rew_mean     | 1.65     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 44       |
|    time_elapsed    | 42       |
|    total_timesteps | 1862     |
| train/             |          |
|    actor_loss      | -0.777   |
|    critic_loss     | 0.000208 |
|    ent_coef 

In [6]:
# Relancer a chaque fois jusqu'a ce qu'il y ait plus le temps, il faut visualiser l'atterissage !
# model.learn(total_timesteps=200_000) # 100k timestep = 1h

In [7]:
testing = gym.make("RocketLander-v0")
testing = RocketLanderWrapper(testing)
testing = FrameSkip(testing, skip=4)
tst = evaluate_policy(model, testing, return_episode_rewards=True)

/home/hanzopgp/miniconda3/envs/deepdac/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [8]:
from numpy import mean, std
rewards = tst[0]
print( "min:",min(rewards) , "max:",max(rewards), "mean:", mean(rewards), "std:", std(rewards) )

min: -1.326755629703257 max: 2.717572975816438 mean: 1.4669136754891952 std: 1.3286637581940766


In [9]:
!apt-get install ffmpeg freeglut3-dev xvfb # For visualization

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [10]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id)])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs, deterministic=True)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [11]:
# record_video('RocketLander-v0', model, video_length=500, prefix='sac-rocket')

(EE) 
Fatal server error:
(EE) Server is already active for display 1
	If this server is no longer running, remove /tmp/.X1-lock
	and start again.
(EE) 


In [12]:
# show_videos('videos', prefix='sac-rocket')

In [13]:
model.save("tqc_rocketlander_"+str(mean(rewards)))